In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# [1-4,时间序列数据建模流程范例](https://lyhue1991.github.io/eat_tensorflow2_in_30_days/1-4,%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97%E6%95%B0%E6%8D%AE%E5%BB%BA%E6%A8%A1%E6%B5%81%E7%A8%8B%E8%8C%83%E4%BE%8B.html)
# https://blog.csdn.net/weixin_45158404/article/details/104720365
# [新冠肺炎Covid-19数据分析](https://zhuanlan.zhihu.com/p/109556102)

In [ ]:
pip install tushare

In [ ]:
import tushare as ts
pro = ts.pro_api("ad41a74271c5e6ba99f5c9dc43c9a1be75bdde38e7e679a732afd7f9")#需要在tushare免费注册一个账号，填入你的token即可调取数据
df = pro.ncov_num(level=2)
#df1 = pro.ncov_num(start_date='20200124',end_date='20200507')#最早是从2020年1月24日开始的数据
df.to_csv('covid19.csv')
df

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow.keras import models,layers,losses,metrics,callbacks

In [ ]:
import os
from glob import glob
result = [y for x in os.walk('/kaggle') for y in glob(os.path.join(x[0], '*'))]
result

In [ ]:
from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

df = pd.read_csv("/kaggle/working/covid19.csv",sep = ",")
df['ann_date']=df['ann_date'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))

In [ ]:
df.plot(x = "ann_date",y = ["confirmed_num","cured_num","dead_num"],figsize=(10,6))
plt.xticks(rotation=60)

In [ ]:
df_data=df.drop(['Unnamed: 0', 'area_name', 'parent_name', 'suspected_num', 'confirmed_num_now', 'suspected_num_now', 'level'], axis=1)
df_data=df_data.set_index("ann_date")
dfdiff=df_data.diff(periods=1).dropna()
dfdiff = dfdiff.reset_index(level="ann_date")
#print(dfdiff)

dfdiff.plot(x = "ann_date",y = ["confirmed_num","cured_num","dead_num"],figsize=(10,6))
plt.xticks(rotation=60)
dfdiff = dfdiff.drop("ann_date",axis = 1).astype("float32")

In [ ]:
dfdiff

In [ ]:
WINDOW_SIZE=8

def batch_dataset(dataset):
    dataset_batched=dataset.batch(WINDOW_SIZE, drop_remainder=True)
    return dataset_batched

ds_data=tf.data.Dataset.from_tensor_slices(tf.constant(dfdiff.values, dtype=tf.float32)) \
.window(WINDOW_SIZE, shift=1).flat_map(batch_dataset)
# ds_data=tf.data.Dataset.from_tensor_slices(tf.constant(dfdiff.values, dtype=tf.float32)) \
# .window(WINDOW_SIZE, shift=1)
# ds_data=ds_data.flat_map(lambda window: window.batch(WINDOW_SIZE))

ds_label=tf.data.Dataset.from_tensor_slices(tf.constant(dfdiff.values[WINDOW_SIZE:], dtype=tf.float32))

ds_train=tf.data.Dataset.zip((ds_data, ds_label)).batch(38).cache()

In [ ]:
print("ds_data", list(ds_data.as_numpy_iterator())[0])
print("ds_label", list(ds_label.as_numpy_iterator())[0])

x, y=next(ds_train.as_numpy_iterator())
print("ds_train", x.shape, y.shape)

# 定义模型

In [ ]:
class Block(layers.Layer):
    def __init__(self, **kwargs):
        super(Block, self).__init__(**kwargs)
        
    def call(self, x_input, x):
        print("Block layer: x_input: ", x_input.shape, x_input[:,-1,:])
        print("Block layer: x: ", x.shape, x)
        x_out=tf.maximum((x)*x_input[:, -1, :], 0.0)
        return x_out
    
    def get_config(self):
        config=super(Block, self).get_config()
        return config

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
x_input=layers.Input(shape=(None, 3), dtype=tf.float32)
x=layers.LSTM(3, return_sequences=True, input_shape=(None,3))(x_input)
x=layers.LSTM(3, return_sequences=True, input_shape=(None,3))(x)
x=layers.LSTM(3, return_sequences=True, input_shape=(None,3))(x)
x=layers.LSTM(3, input_shape=(None,3))(x)
x=layers.Dense(3)(x)

x=Block()(x_input, x)
model=models.Model(inputs=[x_input], outputs=[x])
model.summary()

In [ ]:
class MSPE(losses.Loss):
    def call(self, y_true, y_pred):
        err_percent=(y_true-y_pred)**2/(tf.maximum(y_true**2, 1e-7))
        mean_err_percent=tf.reduce_mean(err_percent)
        return mean_err_percent

    def get_config(self):
        config=super(MSPE, self).get_config()
        return config

In [ ]:
import datetime

optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss=MSPE(name="MSPE"))

logdir = "./data/keras_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tb_callback=tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
lr_callback=tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=100)
stop_callback=tf.keras.callbacks.EarlyStopping(monitor="loss", patience=200)
callbacks_list=[tb_callback, lr_callback, stop_callback]

history=model.fit(ds_train, epochs=500, callbacks=callbacks_list)

In [ ]:
test_input=layers.Input(shape=(4, 3), dtype=tf.float32)
test_input.shape